In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/MIROC6/tasNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/MIROC6/hursNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/MIROC6/uasNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/MIROC6/vasNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')


In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/MIROC6/taNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/MIROC6/zgNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/MIROC6/wapNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/MIROC6/uaNHRegrid_NDJFday_MIROC6_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [5]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
rhD  = rhData.sel(latitude=slice(32,24),longitude=slice(74,86))
tLevD = tLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
uLevD = uLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = wLevData.sel(latitude=slice(32,24),longitude=slice(74,86)).sel(plev=70000,method='nearest')
zLevD = zLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
#w700D

In [6]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10D.vas.values**2)+(u10D.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhD.hurs)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#Calculate inv
inv=t2mD.tas.values-tLevD.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, 
                    coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevD.ua.sel(plev=85000,method='nearest').values-u10D.uas.values)/(zLevD.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, 
                       coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [7]:
AOData = xr.open_dataset('MIROC6-AOindex-NDJF-Daily-1980-2014.nc')

AO=AOData.AO
#datetimeindex = AO.indexes['time'].to_datetimeindex()#
#datetimeindex
#AO['time'] = datetimeindex
AO.values

AO5D=AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

In [8]:
EUData = xr.open_dataset('MIROC6-EUindex-NDJF-Daily-1980-2014.nc')
EU=EUData.EU
#datetimeindex = EU.indexes['time'].to_datetimeindex()
#datetimeindex
#EU['time'] = datetimeindex
EU.values
EU5D=EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


In [9]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [10]:
# Apply Mask 
t2mD.tas.values=t2mD.tas.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.wap.values=w700D.wap.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask

In [11]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).tas
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).wap
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear

In [12]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)

In [13]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

In [14]:
X       = pd.DataFrame([t2m.values,ws.values,rh.values/100.0,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,291.046909,1.347635,0.341624,6.482689,0.034536,0.094668,0.062032,-0.027935
std,2.112291,0.454813,0.111280,0.954333,0.078101,0.115322,0.998374,0.569681
min,284.061432,0.411126,0.116601,1.887170,-0.369412,-0.335652,-3.403053,-1.700168
25%,289.615669,1.033683,0.264722,5.987456,-0.000247,0.019445,-0.606389,-0.428672
50%,291.097870,1.260156,0.317668,6.558190,0.043149,0.095671,0.119816,-0.027952
75%,292.544327,1.573884,0.393941,7.101623,0.082365,0.168105,0.781874,0.370362
max,297.227936,5.448400,0.771439,9.269017,0.287407,0.623184,2.550519,1.757498


In [15]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,-137.819045,-2.963737,-3.070650,-6.794465,-0.442301,-0.821094,-0.062147,0.049048
std,0.473638,2.199721,8.990458,1.048335,12.809859,8.675373,1.002091,1.756177
min,-139.385394,-7.493204,-21.250546,-11.842642,-66.696542,-33.192974,-3.540132,-5.106009
25%,-138.139971,-4.482179,-9.283664,-7.338478,-6.147343,-6.479984,-0.733056,-1.186318
50%,-137.807618,-3.386832,-5.006082,-6.711527,0.970338,-0.745683,-0.004148,0.048995
75%,-137.483280,-1.869475,1.156048,-6.114566,7.402359,4.703356,0.660374,1.276894
max,-136.433077,16.869779,31.654572,-3.733684,41.032820,38.937760,2.435604,5.553068


In [16]:
regLin=load('../../March2021/Observation_models/LRModel.joblib') 

In [17]:
yLR=regLin.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})

In [18]:
dump(y_predLin_ds.yLR,'../Model_plots/MIROC6-LR-Y.joblib')

['../Model_plots/MIROC6-LR-Y.joblib']

In [19]:
regLin2=load('../../March2021/Observation_models/SVRModel.joblib') 
yLR=regLin2.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})
dump(y_predLin_ds.yLR,'../Model_plots/MIROC6-SVR-Y.joblib')

['../Model_plots/MIROC6-SVR-Y.joblib']